In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE87629"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE87629"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE87629.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE87629.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE87629.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from DNA microarray
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: we can use the biopsy data (villus height to crypt depth) as a measure of celiac disease severity
trait_row = 5  # biopsy data, villus height to crypt depth

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert villus height to crypt depth ratio to a continuous value."""
    if not value or value == 'NA' or ':' not in value:
        return None
    
    try:
        # Extract the numeric value after the colon
        parts = value.split(':', 1)
        if len(parts) < 2:
            return None
        
        # Convert to float
        numeric_value = float(parts[1].strip())
        return numeric_value
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Placeholder function for age conversion."""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion."""
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Sample characteristics dictionary from the previous step
    sample_char_dict = {
        0: ['individual: celiac patient A', 'individual: celiac patient C', 'individual: celiac patient G', 'individual: celiac patient H', 'individual: celiac patient K', 'individual: celiac patient L', 'individual: celiac patient M', 'individual: celiac patient N', 'individual: celiac patient O', 'individual: celiac patient P', 'individual: celiac patient Q', 'individual: celiac patient R', 'individual: celiac patient S', 'individual: celiac patient T', 'individual: celiac patient U', 'individual: celiac patient V', 'individual: celiac patient W', 'individual: celiac patient X', 'individual: celiac patient Y', 'individual: celiac patient Z'],
        1: ['disease state: biopsy confirmed celiac disease on gluten-free diet greater than one year'],
        2: ['treatment: control', 'treatment: 6 weeks gluten challenge'],
        3: ['tissue: peripheral whole blood'],
        4: ['cell type: purified pool of B and T cells'],
        5: ['biopsy data, villus height to crypt depth: 2.9', 'biopsy data, villus height to crypt depth: 2.6', 'biopsy data, villus height to crypt depth: 1.1', 'biopsy data, villus height to crypt depth: 0.5', 'biopsy data, villus height to crypt depth: 0.3', 'biopsy data, villus height to crypt depth: 2', 'biopsy data, villus height to crypt depth: 0.4', 'biopsy data, villus height to crypt depth: 2.4', 'biopsy data, villus height to crypt depth: 1.4', 'biopsy data, villus height to crypt depth: 2.7', 'biopsy data, villus height to crypt depth: 3.5', 'biopsy data, villus height to crypt depth: 0.7', 'biopsy data, villus height to crypt depth: 0.2', 'biopsy data, villus height to crypt depth: 2.8', 'biopsy data, villus height to crypt depth: 3', 'biopsy data, villus height to crypt depth: 0.8', 'biopsy data, villus height to crypt depth: 1.2', 'biopsy data, villus height to crypt depth: 1.7', 'biopsy data, villus height to crypt depth: 2.5', 'biopsy data, villus height to crypt depth: 2.1', 'biopsy data, villus height to crypt depth: 3.1'],
        6: ['hybridization batch: 1']
    }
    
    # Convert to a proper format for the geo_select_clinical_features function
    # We need to create a DataFrame where each row is a feature type and columns are samples
    # First, get the maximum number of samples for any feature
    max_samples = max(len(values) for values in sample_char_dict.values())
    
    # Create a DataFrame with features as rows and pad with NaN for missing values
    data = {}
    for row_id, values in sample_char_dict.items():
        data[row_id] = values + [None] * (max_samples - len(values))
    
    clinical_data = pd.DataFrame(data).T  # Transpose to have features as rows
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical features
    preview = preview_df(clinical_features)
    print(f"Clinical features preview: {preview}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to: {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Looking at the identifiers, I can recognize these as Illumina probe IDs (ILMN_xxxxxxx format)
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine the columns for mapping
# Looking at the gene annotation preview, 'ID' seems to contain the probe identifiers (ILMN_xxxxxxx)
# and 'Symbol' contains the gene symbols
prob_col = 'ID'
gene_col = 'Symbol'

print(f"Using columns for mapping: {prob_col} -> {gene_col}")

# 2. Get a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Preview the first few rows of gene expression data
print("Preview of gene expression data:")
preview_gene_data = preview_df(gene_data)
print(preview_gene_data)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"Normalized gene data saved to {out_gene_data_file}")

# Load clinical data from the saved file
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Clinical data shape: {clinical_df.shape}")

# Rename the first column to be our trait
clinical_df = pd.DataFrame({trait: clinical_df.iloc[:, 0]})
print(f"Reformatted clinical data shape: {clinical_df.shape}")
print(f"Clinical data columns: {clinical_df.columns}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
note = "Dataset contains gene expression from B and T cells in celiac disease patients during a gluten challenge. The biopsy data (villus height to crypt depth ratio) is used as a continuous measure of celiac disease severity."

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"Normalized gene data saved to {out_gene_data_file}")

# Debug: examine the structure of clinical_data
print("Clinical data structure:")
print(f"Shape: {clinical_data.shape}")
print("First row of clinical data:")
print(clinical_data.iloc[0])

# Extract clinical features properly
# The first row contains sample accession IDs
sample_ids = clinical_data.iloc[0].values
print(f"Number of sample IDs: {len(sample_ids)}")
print("First few sample IDs:", sample_ids[:5])

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print("Selected clinical data:")
print(f"Shape: {selected_clinical_df.shape}")
print("First few elements:")
print(selected_clinical_df.iloc[:, :5])

# 2. Link the clinical and genetic data using the correct function
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns:", linked_data.columns[:10])  # Show first 10 columns

# Check if we have any samples with trait data
if linked_data.shape[0] > 0 and trait in linked_data.columns:
    # 3. Handle missing values in the linked data
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    if linked_data_clean.shape[0] > 0:
        # 4. Determine whether the trait and demographic features are severely biased
        is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_clean, trait)
        
        # 5. Conduct quality check and save cohort information
        note = "Dataset contains gene expression from B and T cells in celiac disease patients during a gluten challenge. The biopsy data (villus height to crypt depth ratio) is used as a continuous measure of celiac disease severity."
        
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=is_trait_biased,
            df=unbiased_linked_data,
            note=note
        )
        
        # 6. If the linked data is usable, save it as a CSV file
        if is_usable:
            print(f"Data is usable. Saving to {out_data_file}")
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            unbiased_linked_data.to_csv(out_data_file)
        else:
            print("Data is not usable. Not saving linked data file.")
    else:
        print("No samples remaining after cleaning missing values.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=pd.DataFrame(),
            note="No valid samples remained after cleaning. Cannot proceed with analysis."
        )
else:
    print("No trait column in linked data. Cannot proceed with analysis.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to properly link clinical and genetic data. No trait column present in linked data."
    )
    print("Data is not usable. Not saving linked data file.")